In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

In [29]:
# Horizontal and vertical extent of the model in meters:
Lx, Lz = 1600.0e3, 300.0e3

# Number of points in horizontal and vertical direction:
Nx, Nz = 1601, 301

In [30]:
x = np.linspace(0, Lx, Nx)
z = np.linspace(Lz, 0, Nz)

X, Z = np.meshgrid(x, z)

In [31]:
step_initial = 0
step_final = 10000

d_step = 20

In [32]:
from scipy.interpolate import interp1d
import glob
import os
import string
from PIL import Image

In [55]:
def read_density(cont, Nx, Nz):
    '''
    Read density data from density_step.txt to extract interfaces
    '''

    Rho = np.loadtxt("density_"+str(cont)+".txt",skiprows=2, unpack=True, comments="P")
    Rho = np.reshape(Rho, (Nz, Nx))

    return Rho

def extract_interface(z, Z, Nx, Rhoi, rho):
    '''
    Extract interface according to a given density
    '''
    topo_aux = []

    for j in np.arange(Nx):
        topoi = interp1d(z, Rhoi[:,j]) #return a "function" of interpolation to apply in other array
        idx = (np.abs(topoi(Z)-rho)).argmin()
        topo = Z[idx]
        topo_aux = np.append(topo_aux, topo)

    return topo_aux

def extract_topography(step):
    # Read surface topography
    topo= np.loadtxt(f"sp_surface_global_{step}.txt", skiprows=2, unpack=True, comments="P")
    topo = topo/1000
    return topo

def find_nearest(array, value):
    '''Find the index in _array_ nearest to a given _value_'''
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def plot_topography(data, cont, color='purple',cond=False):

    Nx, Nz, Lx, Lz = data

    ##Creating a grid to plot
    xi = np.linspace(0, Lx/1000, Nx)
    zi = np.linspace(-Lz/1000+40, 0+40, Nz) #km, +40 to compensate the air layer above sea level
    xx, zz = np.meshgrid(xi, zi)
    
    z_mean = 40.0 #np.mean(topo[condx])
    
    Rhoi = read_density(cont, Nx, Nz)
    interfaces=[2900, 3365]
    ##Extract layer topography
    z = np.linspace(Lz/1000.0, 0, Nz)
    Z = np.linspace(Lz/1000.0, 0, 8001) #zi
    x = np.linspace(Lx/1000.0, 0, Nx)

    topo_interface = extract_interface(z, Z, Nx, Rhoi, 300.) #200 kg/m3 = air/crust interface
    topo_interface -= np.abs(z_mean) #h_air
    condx = (xi >= 200) & (xi <= 300)
    mean = np.mean(topo_interface[condx])
    topo_interface = topo_interface + np.abs(mean)
    topo_interface = -1.0*topo_interface
    
    ax1.plot(xx[0], topo_interface, '-', color=f'xkcd:{color}', label='density')
#      cond if file has surface_tracking    
    if cond:
        topo = 40 + extract_topography(cont)
        mean = np.mean(topo[condx])
        topo = topo - np.abs(mean)
        ax1.plot(xx[0], topo, color="b", label=f"surface")
    return 0

Load the parameter file to generate the grid of the model:

In [56]:
with open("param.txt", "r") as f:
    line = f.readline()
    line = line.split()
    Nx = int(line[2])
    line = f.readline()
    line = line.split()
    Nz = int(line[2])
    line = f.readline()
    line = line.split()
    Lx = float(line[2])
    line = f.readline()
    line = line.split()
    Lz = float(line[2])

print(
    "nx:", Nx, "\n",
    "nz:", Nz, "\n",
    "Lx:", Lx, "\n",
    "Lz:", Lz
)

data = [Nx, Nz, Lx, Lz]

nx: 1601 
 nz: 301 
 Lx: 1600000.0 
 Lz: 300000.0


Create the grid in kilometers:

In [57]:
xi = np.linspace(0, Lx / 1e3, Nx)
zi = np.linspace(-Lz / 1e3, 0, Nz)

xx, zz = np.meshgrid(xi, zi)

In [58]:
# Define the thickness of the air layer in kilometers
thickness_air = 40.0

Plot the results:

In [ ]:
xlims = [0, 1600]
ylims = [-2, 1]



for cont in range(step_initial, step_final + d_step, d_step):  
    
    # Read time
    time = np.loadtxt("time_" + str(cont) + ".txt", dtype="str")
    time = time[:, 2:]
    time = time.astype("float")
   
    # Read density
    rho = pd.read_csv(
        "density_" + str(cont) + ".txt",
        delimiter=" ",
        comment="P",
        skiprows=2,
        header=None,
    )
    rho = rho.to_numpy()
    rho[np.abs(rho) < 1.0e-200] = 0
    rho = np.reshape(rho, (Nx, Nz), order="F")
    rho = np.transpose(rho)
    
    # Read temperature
    temper = pd.read_csv(
        "temperature_" + str(cont) + ".txt",
        delimiter=" ",
        comment="P",
        skiprows=2,
        header=None,
    )
    temper = temper.to_numpy()
    
    temper = np.reshape(temper, (Nx, Nz), order="F")
    temper = np.transpose(temper)
    
    # Read strain
    strain = pd.read_csv(
        "strain_" + str(cont) + ".txt",
        delimiter=" ",
        comment="P",
        skiprows=2,
        header=None,
    )
    strain = strain.to_numpy()
    strain[np.abs(strain) < 1.0e-200] = 0
    strain = np.reshape(strain, (Nx, Nz), order="F")
    strain = np.transpose(strain)
    strain[rho < 200] = 0
    strain_log = np.log10(strain)
    
    
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex = True, gridspec_kw={'height_ratios': [1, 2],}, figsize=(15, 5))
    ax1.set_title("Time = %.1lf Myr\n\n" % (time[0] / 1.0e6))
    plot_topography(data, cont, color='brown', cond=True)
    ax1.grid('-k', alpha=0.7)
    ax1.set_ylabel('Depth [km]')
    ax1.tick_params(axis='y')
    ax1.tick_params(bottom = False)
    ax1.set_xlim(xlims)
    ax1.set_ylim(ylims)
    ax1.legend()

    # Create the colors to plot the density
    cr = 255.0
    color_upper_crust = (228.0 / cr, 156.0 / cr, 124.0 / cr)
    color_lower_crust = (240.0 / cr, 209.0 / cr, 188.0 / cr)
    color_lithosphere = (155.0 / cr, 194.0 / cr, 155.0 / cr)
    color_asthenosphere = (207.0 / cr, 226.0 / cr, 205.0 / cr)
    colors = [
        color_upper_crust, 
        color_lower_crust, 
        color_lithosphere, 
        color_asthenosphere
    ]
    # Plot density
    ax2.contourf(
        xx,
        zz + thickness_air,
        rho,
        levels=[200.0, 2750, 2900, 3365, 3900],
        colors=colors,
    )  
    
    ax2.contour(
        xx,
        zz + thickness_air,
        temper,
        levels=[400.0, 800, 1000, 1200],
        colors="r"
    ) 

    # Plot strain_log
    
    ax2.imshow(
        strain_log[::-1, :],
        extent=[0, Lx / 1e3, -Lz / 1e3 + thickness_air, thickness_air],
        zorder=100,
        alpha=0.2,
        cmap=plt.get_cmap("Greys"),
        vmin=-0.5,
        vmax=0.9,
    )
    ax2.set_xlabel("x [km]")
    ax2.set_ylabel("Depth [km]")
    
    
    b1 = [0.74, 0.31, 0.15, 0.15]
    bv1 = plt.axes(b1)

    A = np.zeros((100, 10))

    A[:25, :] = 2700
    A[25:50, :] = 2800
    A[50:75, :] = 3300
    A[75:100, :] = 3400

    A = A[::-1, :]

    xA = np.linspace(-0.5, 0.9, 10)
    yA = np.linspace(0, 1.5, 100)

    xxA, yyA = np.meshgrid(xA, yA)
    air_threshold = 200
    plt.contourf(
        xxA,
        yyA,
        A,
        levels=[air_threshold, 2750, 2900, 3365, 3900],
        colors=colors,
    )

    bv1.imshow(
        xxA[::-1, :],
        extent=[-0.5, 0.9, 0, 1.5],
        zorder=100,
        alpha=0.2,
        cmap=plt.get_cmap("Greys"),
        vmin=-0.5,
        vmax=0.9,
    )

    bv1.set_yticklabels([])
    bv1.tick_params(axis='x', labelsize=8)
    
    bv1.set_xlabel("$log_{10}(\epsilon_{II})$", size=18)
    
    plt.subplots_adjust(hspace=0)
    
    fig.canvas.draw()
    buf = fig.canvas.tostring_rgb()
    width, height = fig.canvas.get_width_height()
    pil_image = Image.frombytes("RGB", (width, height), buf)
    
    pil_image.save('im_%05d.png'%cont)
    
    #plt.show()